## 최종적으로 user에게 추천된 영화와 그 user가 시청한 영화들과의 비교

- 직접 눈으로 확인하였다.

#### EDA 결론

1. 시리즈 영화 추천에 대해 다시 고민하게 되었다.
2. 각 모델을 ensemble 하기 전, 각 모델에서 추천한 영화들과 추천 받은 user와의 유사도 적용에 대해 고민해보게 되었다.

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import json

In [2]:
ens_result = pd.read_csv('RecVAE_ensemble/ens_result.csv')
dae_result = pd.read_csv('RecVAE_ensemble/dae_result.csv')
vae_result = pd.read_csv('RecVAE_ensemble/vae_result.csv')
hvamp_result = pd.read_csv('RecVAE_ensemble/hvamp_result.csv')
recvae_result = pd.read_csv('RecVAE_ensemble/recvae_result.csv')

raw_data = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv')


In [3]:
inter_differ_df = pd.read_csv('output_inter_differ_df.csv')
dae_lowest_df = pd.read_csv('/opt/ml/input/code/each_model_eda/dae_lowest_result_users.csv')
vae_lowest_df = pd.read_csv('/opt/ml/input/code/each_model_eda/vae_lowest_result_users.csv')
hvamp_lowest_df = pd.read_csv('/opt/ml/input/code/each_model_eda/hvamp_lowest_user.csv')
recvae_lowest_df = pd.read_csv('/opt/ml/input/code/each_model_eda/recvae_lowest_user.csv')

In [4]:
item_info_list_df = pd.read_csv('/opt/ml/input/code/each_model_eda/item_info_list_df.csv')

In [5]:
item_info_list_df

,title,genre,director
0,['Planet of the Apes (2001)'],"['Drama', 'Adventure', 'Action', 'science_fict...",['nm0000318']
1,['Hackers (1995)'],"['Crime', 'Adventure', 'Thriller', 'Action']",['nm0812200']
2,"['Secret Garden, The (1993)']","['Drama', 'Children']",['nm0002140']
3,"['Aristocats, The (1970)']","['Animation', 'Children']",['nm0718627']
4,"['Dark Crystal, The (1982)']","['Adventure', 'Fantasy']","['nm0000568', 'nm0001345']"
...,...,...,...
6802,['Poison Ivy (1992)'],"['Drama', 'Thriller']",[nan]
6803,"['Other Woman, The (2014)']","['Romance', 'Comedy']",['nm0001024']
6804,"['Rebound, The (2009)']","['Romance', 'Comedy']",[nan]
6805,['Selma (2014)'],['Drama'],[nan]


In [214]:
ens_result

,user,item
0,11,2174
1,11,4370
2,11,2
3,11,58025
4,11,37386
...,...,...
313595,138493,33615
313596,138493,32587
313597,138493,110
313598,138493,2012


inter_differ_df 파일은 ensemble에 사용한 각 모델들의 결과와 최종적으로 ensemble한 모델의 결과와의 교집합과 차집합을 나타내고 있는 DataFrame이다.

In [124]:
inter_differ_df

,inter_all,inter_ens_dae,inter_ens_vae,inter_ens_hvamp,inter_ens_recvae,differ_dae,differ_vae,differ_hvamp,differ_recvae
0,"[""['Star Wars: Episode IV - A New Hope (1977)'...","[""['Full Metal Jacket (1987)']"", ""['Drama', 'W...","[""['City of God (Cidade de Deus) (2002)']"", ""[...","[""['Full Metal Jacket (1987)']"", ""['Drama', 'W...","[""['Full Metal Jacket (1987)']"", ""['Drama', 'W...","[""['Garden State (2004)']"", ""['Drama', 'Romanc...","[""['To Catch a Thief (1955)']"", ""['Crime', 'Ro...","[""['Dog Day Afternoon (1975)']"", ""['Crime', 'D...","[""['Annie Hall (1977)']"", ""['Romance', 'Comedy..."
1,"[""['Resident Evil (2002)']"", ""['science_fictio...","[""['Pitch Black (2000)']"", ""['Thriller', 'Horr...","[""['Pitch Black (2000)']"", ""['Thriller', 'Horr...","[""['Alien (1979)']"", ""['Horror', 'science_fict...","[""['Pitch Black (2000)']"", ""['Thriller', 'Horr...","[""['Thing, The (1982)']"", ""['Action', 'Thrille...","[""['Crow, The (1994)']"", ""['Crime', 'Fantasy',...","[""['Crow, The (1994)']"", ""['Crime', 'Fantasy',...","[""['Lord of the Rings: The Return of the King,..."
2,"[""['Pulp Fiction (1994)']"", ""['Crime', 'Drama'...","[""['Catch Me If You Can (2002)']"", ""['Drama', ...","[""['The Butterfly Effect (2004)']"", ""['Drama',...","[""['Catch Me If You Can (2002)']"", ""['Drama', ...","[""['Catch Me If You Can (2002)']"", ""['Drama', ...","[""['Godfather: Part II, The (1974)']"", ""['Dram...","[""['Lord of the Rings: The Fellowship of the R...","[""['Ring, The (2002)']"", ""['Mystery', 'Thrille...","[""['Shrek 2 (2004)']"", ""['Animation', 'Childre..."
3,"[""['Pulp Fiction (1994)']"", ""['Crime', 'Drama'...","[""['Goodfellas (1990)']"", ""['Drama', 'Crime']""...","[""['Fight Club (1999)']"", ""['Crime', 'Thriller...","[""['Fight Club (1999)']"", ""['Crime', 'Thriller...","[""['Fight Club (1999)']"", ""['Crime', 'Thriller...","[""['Casino (1995)']"", ""['Drama', 'Crime']"", ""[...","[""['Slumdog Millionaire (2008)']"", ""['Drama', ...","[""['Green Mile, The (1999)']"", ""['Crime', 'Dra...","[""['Inside Man (2006)']"", ""['Crime', 'Thriller..."
4,"[""['Forrest Gump (1994)']"", ""['Drama', 'Romanc...","['[""King\'s Speech, The (2010)""]', ""['Drama']""...","['[""King\'s Speech, The (2010)""]', ""['Drama']""...","['[""King\'s Speech, The (2010)""]', ""['Drama']""...","['[""King\'s Speech, The (2010)""]', ""['Drama']""...","[""['Indiana Jones and the Last Crusade (1989)'...","[""['Toy Story 3 (2010)']"", ""['Animation', 'Chi...","[""['Indiana Jones and the Last Crusade (1989)'...","[""['Argo (2012)']"", ""['Drama', 'Thriller']"", ""..."
...,...,...,...,...,...,...,...,...,...
31355,"[""['Lord of the Rings: The Two Towers, The (20...","[""['Shrek (2001)']"", ""['Animation', 'Children'...","[""['Shrek (2001)']"", ""['Animation', 'Children'...","[""['Shrek (2001)']"", ""['Animation', 'Children'...","[""['Lord of the Rings: The Two Towers, The (20...","[""['L.A. Confidential (1997)']"", ""['Crime', 'M...","['[""Ocean\'s Eleven (2001)""]', ""['Crime', 'Thr...","[""['Platoon (1986)']"", ""['Drama', 'War']"", ""['...","[""['Taxi Driver (1976)']"", ""['Crime', 'Thrille..."
31356,"[""['Truman Show, The (1998)']"", ""['Drama', 'Co...","[""['Goodfellas (1990)']"", ""['Drama', 'Crime']""...","[""['Inside Man (2006)']"", ""['Crime', 'Thriller...","[""['Goodfellas (1990)']"", ""['Drama', 'Crime']""...","[""['Goodfellas (1990)']"", ""['Drama', 'Crime']""...","[""['Joe Versus the Volcano (1990)']"", ""['Roman...","[""['Three Burials of Melquiades Estrada, The (...","[""['History of Violence, A (2005)']"", ""['Crime...","[""['Babel (2006)']"", ""['Drama', 'Thriller']"", ..."
31357,"[""['Pulp Fiction (1994)']"", ""['Crime', 'Drama'...","[""['Pulp Fiction (1994)']"", ""['Crime', 'Drama'...","[""['Army of Darkness (1993)']"", ""['Adventure',...","[""['Pulp Fiction (1994)']"", ""['Crime', 'Drama'...","[""['Army of Darkness (1993)']"", ""['Adventure',...","[""['Django Unchained (2012)']"", ""['Drama', 'We...","[""['Children of

위 DataFrame을 .csv파일로 직접 눈으로 확인한 결과, 종종 최종적으로 ensemble하여 나온 결과들이 나머지 model의 추천 결과와 상이한 경향을 보이는 경우가 있었다.

In [ ]:
raw_data_item = raw_data.groupby('item')

In [ ]:
item_info_list_df['views'] =  raw_data_item.size().values

In [125]:
with open('show2id.json', 'r', encoding="utf-8") as f:
    show2id = json.load(f)

with open('profile2id.json', 'r', encoding="utf-8") as f:
    profile2id = json.load(f)

In [126]:
def user_numerize(tp):
    sid = tp['user'].apply(lambda x: profile2id[str(x)])
    return sid

In [127]:
def numerize_for_infer(tp):
    uid = tp['user'].apply(lambda x: profile2id[str(x)])
    sid = tp['item'].apply(lambda x: show2id[str(x)])
    return pd.DataFrame(data={'uid': uid, 'sid': sid}, columns=['uid', 'sid'])

In [171]:
raw_data

,user,item,time
0,11,4643,1230782529
1,11,170,1230782534
2,11,531,1230782539
3,11,616,1230782542
4,11,2140,1230782563
...,...,...,...
5154466,138493,44022,1260209449
5154467,138493,4958,1260209482
5154468,138493,68319,1260209720
5154469,138493,40819,1260209726


In [172]:
raw_data = numerize_for_infer(raw_data)
raw_data

,uid,sid
0,11825,0
1,11825,1
2,11825,2
3,11825,3
4,11825,4
...,...,...
5154466,10783,477
5154467,10783,1325
5154468,10783,331
5154469,10783,558


In [173]:
raw_data.columns = ['user','item']
raw_data

,user,item
0,11825,0
1,11825,1
2,11825,2
3,11825,3
4,11825,4
...,...,...
5154466,10783,477
5154467,10783,1325
5154468,10783,331
5154469,10783,558


In [128]:
dae_lowest_df = dae_lowest_df.drop('Unnamed: 0',axis=1)
dae_lowest_df.columns = ['user']

In [129]:
dae_lowest_df['user'] = user_numerize(dae_lowest_df)
dae_lowest_df

,user
0,16432
1,5095
2,2816
3,26467
4,11565
...,...
5250,9005
5251,19625
5252,12802
5253,23437


In [130]:
dae_lowest_df.sort_values('user').values.flatten()

array([    2,     4,    11, ..., 31352, 31353, 31354])

In [131]:
vae_lowest_df = vae_lowest_df.drop('Unnamed: 0',axis=1)
vae_lowest_df.columns = ['user']
vae_lowest_df

,user
0,43
1,72
2,77
3,147
4,163
...,...
5518,138280
5519,138305
5520,138397
5521,138470


In [132]:
vae_lowest_df['user'] = user_numerize(vae_lowest_df)
vae_lowest_df

,user
0,5095
1,20527
2,2816
3,25563
4,11565
...,...
5518,4523
5519,9005
5520,19625
5521,23437


In [133]:
hvamp_lowest_df

,user
0,43
1,72
2,77
3,91
4,102
...,...
4313,138305
4314,138397
4315,138456
4316,138459


In [134]:
hvamp_lowest_df['user'] = user_numerize(hvamp_lowest_df)
hvamp_lowest_df

,user
0,5095
1,20527
2,2816
3,14515
4,28255
...,...
4313,9005
4314,19625
4315,17277
4316,26934


In [135]:
recvae_lowest_df

,user
0,43
1,72
2,77
3,91
4,102
...,...
4732,138305
4733,138397
4734,138456
4735,138459


In [136]:
recvae_lowest_df['user'] = user_numerize(recvae_lowest_df)
recvae_lowest_df
#len(recvae_df.groupby('user').indices)

,user
0,5095
1,20527
2,2816
3,14515
4,28255
...,...
4732,9005
4733,19625
4734,17277
4735,26934


In [215]:
ens_result_df = numerize_for_infer(ens_result)

In [216]:
ens_result_df

,uid,sid
0,11825,694
1,11825,805
2,11825,1845
3,11825,540
4,11825,535
...,...,...
313595,10783,472
313596,10783,591
313597,10783,205
313598,10783,74


In [137]:
dae_lowest_idx = dae_lowest_df.sort_values('user').values.flatten()
vae_lowest_idx = vae_lowest_df.sort_values('user').values.flatten()
hvamp_lowest_idx = hvamp_lowest_df.sort_values('user').values.flatten()
recvae_lowest_idx = recvae_lowest_df.sort_values('user').values.flatten()


In [138]:
print(len(set(dae_lowest_idx)))
print(len(set(hvamp_lowest_idx)))

5255
4318


In [147]:
type(recvae_lowest_idx)

numpy.ndarray

In [139]:
all = list(set(dae_lowest_idx) & set(vae_lowest_idx) & set(recvae_lowest_idx))

In [148]:
type(all)

list

In [206]:
fin_dae_lowest_df =inter_differ_df.loc[dae_lowest_idx]
fin_vae_lowest_df =inter_differ_df.loc[vae_lowest_idx]
fin_hvamp_lowest_df = inter_differ_df.loc[hvamp_lowest_idx]
fin_recvae_lowest_df = inter_differ_df.loc[recvae_lowest_idx]

In [142]:
fin_all_lowest_df = inter_differ_df.loc[all]

In [149]:
all[13]

38

In [143]:
fin_dae_lowest_df.to_csv('fin_dae_lowest_df.csv',index=False)
fin_vae_lowest_df.to_csv('fin_vae_lowest_df.csv',index=False)
fin_hvamp_lowest_df.to_csv('fin_hvamp_lowest_df.csv',index=False)
fin_recvae_lowest_df.to_csv('fin_recvae_lowest_df.csv',index=False)

In [144]:
fin_all_lowest_df.to_csv('fin_all_lowest_df.csv',index=False)

In [152]:
fin_all_lowest_df['inter_all']

24579    ["['Lord of the Rings: The Fellowship of the R...
16387    ["['Fight Club (1999)']", "['Crime', 'Thriller...
16389    ["['Pulp Fiction (1994)']", "['Crime', 'Drama'...
24584    ["['Bourne Identity, The (1988)']", "['Adventu...
24586    ["['X-Men (2000)']", "['science_fiction', 'Adv...
                               ...                        
24559    ["['Departed, The (2006)']", "['Crime', 'Thril...
24567    ["['Sideways (2004)']", "['Drama', 'Romance', ...
24568    ["['Fight Club (1999)']", "['Crime', 'Thriller...
8186     ["['City of God (Cidade de Deus) (2002)']", "[...
24574    ["['Forrest Gump (1994)']", "['Drama', 'Romanc...
Name: inter_all, Length: 3117, dtype: object

In [170]:
all[11]

16418

In [167]:
fin_all_lowest_df['inter_all'].iloc[11]

'["[\'Shawshank Redemption, The (1994)\']", "[\'Crime\', \'Drama\']", "[\'nm0001104\']"]'

In [181]:
fin_all_lowest_df['inter_all'].iloc[32]


'["[\'Fight Club (1999)\']", "[\'Crime\', \'Thriller\', \'Drama\', \'Action\']", "[\'nm0000399\']", "[\'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1964)\']", "[\'War\', \'Comedy\']", "[\'nm0000040\']", "[\'Godfather, The (1972)\']", "[\'Drama\', \'Crime\']", "[\'nm0000338\']", "[\'American Beauty (1999)\']", "[\'Drama\', \'Comedy\']", "[\'nm0005222\']", "[\'Eternal Sunshine of the Spotless Mind (2004)\']", "[\'Drama\', \'Romance\', \'science_fiction\']", "[\'nm0327273\']", "[\'Matrix, The (1999)\']", "[\'science_fiction\', \'Thriller\', \'Action\']", "[\'nm0905152\', \'nm0905154\']"]'

In [203]:
all[11]

16418

In [179]:
raw_data[raw_data['user']==16418]['item'].values

array([  84,  271,   85,   82,  285,  446, 1173, 1210, 4660, 1567, 2310,
        205,  590,  287,  159,  265,  286,  925,  439,  160, 1222, 2107,
       1066,  597,  268, 1571,  847, 1574,  297,  591,  487,  450,  300,
        298,  130,  726,   60, 1177,  149,  307,  360, 3686, 3447, 2211,
       1044, 1075,  536, 5075, 1935, 1740, 3410])

In [219]:
ens_result_df.columns = ['user','item']

In [221]:
ens_result_df[ens_result_df['user']==16418]['item'].values

array([313, 198, 483, 200, 289, 292,  72, 374, 267, 482])

In [222]:
item_info_list_df.loc[ens_result_df[ens_result_df['user']==16418]['item'].values]

,title,genre,director,views
313,"['Dark Knight, The (2008)']","['Crime', 'Drama', 'Action']",['nm0634240'],13060
198,"['Shawshank Redemption, The (1994)']","['Crime', 'Drama']",['nm0001104'],18168
483,['Avatar (2009)'],"['Adventure', 'Action', 'science_fiction']",['nm0000116'],6113
200,['Lord of the Rings: The Fellowship of the Rin...,"['Adventure', 'Fantasy']",['nm0001392'],17237
289,['Shrek (2001)'],"['Animation', 'Children', 'Adventure', 'Romanc...","['nm0421776', 'nm0011470']",13495
292,"['Lord of the Rings: The Two Towers, The (2002)']","['Adventure', 'Fantasy']",['nm0001392'],16387
72,['Back to the Future (1985)'],"['Adventure', 'Comedy', 'science_fiction']",['nm0000709'],13541
374,"['Lord of the Rings: The Return of the King, T...","['Drama', 'Adventure', 'Fantasy', 'Action']",['nm0001392'],16656
267,['Toy Story (1995)'],"['Animation', 'Children', 'Adventure', 'Fantas...",['nm0005124'],12217
482,['Harry Potter and the Goblet of Fire (2005)'],"['Adventure', 'Fantasy', 'Thriller']",['nm0001565'],6494


In [202]:
item_info_list_df.loc[raw_data[raw_data['user']==16418]['item'].values]

,title,genre,director,views
84,['Star Wars: Episode IV - A New Hope (1977)'],"['Adventure', 'Action', 'science_fiction']",['nm0000184'],14613
271,['Raiders of the Lost Ark (Indiana Jones and t...,"['Adventure', 'Action']",['nm0000229'],14224
85,['Star Wars: Episode V - The Empire Strikes Ba...,"['science_fiction', 'Adventure', 'Action']",['nm0449984'],14401
82,"['Matrix, The (1999)']","['science_fiction', 'Thriller', 'Action']","['nm0905152', 'nm0905154']",19699
285,['Indiana Jones and the Last Crusade (1989)'],"['Adventure', 'Action']",['nm0000229'],11156
446,['Despicable Me (2010)'],"['Crime', 'Animation', 'Comedy', 'Children']","['nm1853544', 'nm0719208']",1792
1173,['Argo (2012)'],"['Drama', 'Thriller']",['nm0000255'],1854
1210,['Gravity (2013)'],"['Action', 'science_fiction']",['nm0190859'],1635
4660,['Shivers (They Came from Within) (1975)'],"['Drama', 'Horror', 'science_fiction']",['nm0000343'],376
1567,['Interstellar (2014)'],['science_fiction'],['nm0634240'],1013


In [223]:
item_info_list_df.loc[ens_result_df[ens_result_df['user']==8278]['item'].values]


,title,genre,director,views
427,"['Godfather, The (1972)']","['Drama', 'Crime']",['nm0000338'],13338
269,['Fargo (1996)'],"['Crime', 'Comedy', 'Thriller', 'Drama']","['nm0001053', 'nm0001054']",10396
271,['Raiders of the Lost Ark (Indiana Jones and t...,"['Adventure', 'Action']",['nm0000229'],14224
46,['American Beauty (1999)'],"['Drama', 'Comedy']",['nm0005222'],15213
713,['Dr. Strangelove or: How I Learned to Stop Wo...,"['War', 'Comedy']",['nm0000040'],7617
260,['Fight Club (1999)'],"['Crime', 'Thriller', 'Drama', 'Action']",['nm0000399'],18437
82,"['Matrix, The (1999)']","['science_fiction', 'Thriller', 'Action']","['nm0905152', 'nm0905154']",19699
688,"[""One Flew Over the Cuckoo's Nest (1975)""]",['Drama'],['nm0001232'],10603
80,['Eternal Sunshine of the Spotless Mind (2004)'],"['Drama', 'Romance', 'science_fiction']",['nm0327273'],13292
40,['Alien (1979)'],"['Horror', 'science_fiction']",['nm0000631'],10107


In [201]:
item_info_list_df.loc[raw_data[raw_data['user']==8278]['item'].values]


,title,genre,director,views
1027,"[""Rosemary's Baby (1968)""]","['Drama', 'Thriller', 'Horror']",['nm0000591'],2151
1810,['Arlington Road (1999)'],['Thriller'],['nm0671210'],1162
828,"['Dirty Dozen, The (1967)']","['Drama', 'War', 'Action']",['nm0000736'],1808
2667,['Dead Man (1995)'],"['Drama', 'Western', 'Mystery']",['nm0000464'],1078
1535,['Mystery Science Theater 3000: The Movie (199...,"['Comedy', 'science_fiction']",[nan],906
1435,"['Jerk, The (1979)']",['Comedy'],['nm0005348'],1489
3110,['Nell (1994)'],['Drama'],['nm0000776'],1169
550,"['Producers, The (1968)']",['Comedy'],['nm0000316'],1445
1801,['Fletch (1985)'],"['Crime', 'Mystery', 'Comedy']",['nm0006916'],1007
3052,"[""Man Bites Dog (C'est arrivé près de chez vou...","['Crime', 'Comedy', 'Thriller', 'Drama']",[nan],969
